# Data cleaning Immobiliare.it

Inizio con importare le librerie utili

In [693]:
import numpy as np
import pandas as pd
import re

Controllo il dataframe grezzo nelle sue dimensioni e valori

In [694]:
df_raw = pd.read_csv('house_prices_italy.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price
0,0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,2,abruzzo,Silvi,125m²,4,3,€ 165.000
3,3,abruzzo,Pescara,227m²,5,3+,€ 299.000
4,4,abruzzo,Pescara,126m²,5+,2,"€ 139.000€ 149.000(-6,7%)"


In [695]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39949 entries, 0 to 39948
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  39949 non-null  int64 
 1   region      39949 non-null  object
 2   city        39949 non-null  object
 3   area        39914 non-null  object
 4   rooms       38962 non-null  object
 5   toilets     39949 non-null  object
 6   price       39949 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


Vedo se ci sono valori nulli

In [696]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           35
rooms         987
toilets         0
price           0
dtype: int64

Ho deciso di togliere la prima colonna visot che non mi da alcuna informaione utile

In [697]:
df=df_raw.drop("Unnamed: 0", axis=1)

In [698]:
df.head()

,region,city,area,rooms,toilets,price
0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,abruzzo,Silvi,125m²,4,3,€ 165.000
3,abruzzo,Pescara,227m²,5,3+,€ 299.000
4,abruzzo,Pescara,126m²,5+,2,"€ 139.000€ 149.000(-6,7%)"


Devo forzare la colonna 'Area' ad essere di tipo stringa, altrimenti dopo mi restituisce errore

In [699]:
df['area'] = df['area'].astype(str)

Rinomino le colonne in maniera tale che posseggano le dimensioni della vairabile che contengono

In [700]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)
df.sample(10)

,region,city,area[m2],rooms,toilets,price[€]
18487,marche,Fano,149m²,5+,2,€ 340.000
391,abruzzo,Tortoreto,54m²,3,1,€ 158.000
24204,puglia,Fasano,147m²,5+,2,€ 950.000
22717,piemonte,Torino,53m²,2,1,€ 34.900
38523,veneto,Negrar di Valpolicella,138m²,5,2,€ 263.000
26819,sardegna,Busachi,73m²,2,1,€ 80.000
22234,piemonte,Torino,83m²,3,1,€ 138.000
30494,toscana,Firenze,82m²,4,1,€ 299.000
28457,sicilia,Siracusa,40m²,2,1,€ 67.000
37068,valle-d-aosta,La Salle,195m²,5,3,€ 590.000


Inizio dal pulire la colonna riferita all'area degli appartamenti. Utilizzo per questo le Regex

In [701]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89,3,1,€ 75.000
1,abruzzo,Spoltore,199,5+,3+,€ 235.000
2,abruzzo,Silvi,125,4,3,€ 165.000
3,abruzzo,Pescara,227,5,3+,€ 299.000
4,abruzzo,Pescara,126,5+,2,"€ 139.000€ 149.000(-6,7%)"


In [702]:
df.isna().sum()

region        0
city          0
area[m2]     35
rooms       987
toilets       0
price[€]      0
dtype: int64

In alcuni casi la Regex non ha trovato un valore ma una stringa "Prezzo su richiesta", li considero quindi come null

Procedo ora con il pulire la colonna dei prezzi

In [703]:
p_price = re.compile('[0-9]+\.[0-9]+')

In [704]:
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if p_price.search(x) is None else p_price.search(x).group())
df.isna().sum()

region         0
city           0
area[m2]      35
rooms        987
toilets        0
price[€]    1235
dtype: int64

In [705]:
df.region.unique()

array(['abruzzo', 'basilicata', 'campania', 'calabria', 'emilia-romagna',
       'friuli-venezia-giulia', 'lazio', 'liguria', 'lombardia', 'marche',
       'molise', 'piemonte', 'puglia', 'sardegna', 'sicilia', 'toscana',
       'trentino-alto-adige', 'umbria', 'valle-d-aosta', 'veneto'],
      dtype=object)

In [706]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      35
rooms        987
toilets        0
price[€]    1235
dtype: int64